In [ ]:
setwd("../../../../")

In [ ]:
RhpcBLASctl::blas_set_num_threads(1)
library(Seurat)
library(dplyr)
#library(tidyverse)
library(here)
#library(readxl)
#library(Matrix)
library(IMSBSeuratHelpers)
library(ggplot2)
#library(sctransform)
library(future)
library(data.table)
library(xlsx)
plan("multicore", workers = 1)
options(future.globals.maxSize = 20 * 1000 * 1024^2)#20GB
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)

In [ ]:
#library(devtools)
#install_github("immunogenomics/harmony")

In [ ]:
samples_ID="Harmony_Ustekinumab_4PK4PB_T_CD4EMRM"

In [ ]:
seu<-readRDS(paste0("Integrated_objects/Harmony_Ustekinumab_4PK4PB_T_CD4EMRM_clustered.rds"))

In [ ]:
DefaultAssay(seu)<-'RNA'

In [ ]:
seu

In [ ]:
table(seu@meta.data$tissue)

In [ ]:
as.data.frame(table(seu@meta.data$sample))

In [ ]:
as.data.frame(table(seu@meta.data$patient))

In [ ]:
#DimPlot(object = seu, reduction = 'tsne',label = F, 
#        pt.size = 0.1)+ theme(aspect.ratio=1)
DimPlot(object = seu, reduction = 'umap',label = T, 
        pt.size = 0.1)+ theme(aspect.ratio=1)

In [ ]:
DimPlot(object = seu, reduction = 'umap',label = F, 
        group.by = "tissue",
        pt.size = 0.1)+ theme(aspect.ratio=1)

In [ ]:
DimPlot(object = seu, reduction = 'umap',label = F, 
        group.by = "patient",
        pt.size = 0.1)+ theme(aspect.ratio=1)

In [ ]:
DimPlot(object = seu, reduction = 'umap',label = F, 
        group.by = "sample",
        pt.size = 0.1)+ theme(aspect.ratio=1)

In [ ]:
new.cluster.ids <- c(
'CD4TEM',
'CD4TEM',
'TFH',
'TRM1',
'TRM17',
'TFH'
)
names(new.cluster.ids) <- levels(seu)
seu_1 <- RenameIdents(seu, new.cluster.ids)

In [ ]:
DimPlot(object = seu_1, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T
       ) + theme(aspect.ratio=1,legend.position = "right")

In [ ]:
DimPlot(object = seu_1, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 6, repel = T
       ) + theme(aspect.ratio=1,legend.position = "right")+NoLegend()

In [ ]:
seu_1$cell_type <- Idents(seu_1)
head(seu_1@meta.data)
Idents(seu_1)<-"cell_type"
levels(seu_1)

In [ ]:
my_levels <- c(
'TRM17',
'TRM1',
'CD4TEM',
'TFH'
)
levels(seu_1) <- my_levels

In [ ]:
DimPlot(object = seu_1, reduction = 'umap',label = TRUE, 
        pt.size = 0.1,label.size = 4, repel = T
       ) + theme(aspect.ratio=1,legend.position = "right")

In [ ]:
levels(seu_1)

In [ ]:
cols = c(
'orange',#'TRM17',
'red',#'TRM1',
'#b0afd1',#'CD4TEM',
'cadetblue1'#'TFH'
)
#levels(seu_1) <- my_levels

In [ ]:
DimPlot(object = seu_1, reduction = 'umap',label = TRUE,
        cols = cols,pt.size = 0.1,label.size = 6, repel = T
       ) + theme(aspect.ratio=1,legend.position = "right")

In [ ]:
seu<-seu_1

In [ ]:
#...#

In [ ]:
#saveRDS(seu_1, file = paste0("Integrated_objects/",samples_ID,"_clean.rds"))

In [ ]:
human_Trm_gene_up<-c("CD69","CA10","ITGA1","ITGAE","IL2","IL10","CXCR6","CXCL13","KCNK5","RGS1","CRTAM","DUSP6","PDCD1","IL23R")    
human_Trm_gene_down<-c("STK38","TTC16","SELL","KLF3","KLF2","D4S234E","SBK1","FAM65B","TTYH2","NPDC1","KRT72","S1PR1","SOX13","KRT73","TSPAN18","PTGDS","RAP1GAP2","CX3CR1")
human_Trm_gene_up_expressed <- intersect(human_Trm_gene_up, rownames(seu))
human_Trm_gene_down_expressed <- intersect(human_Trm_gene_down, rownames(seu))


human_CD4_CD69_up<-c("CD69","CA10","IL17F","IL2","CDHR1","IL21","IL10","IL23R","CXCL13","CXCR6","KCNK5","ITGA1","JAG2","SRGAP3","TOX2","CH25H","NEK10","TMEM200A","MYO1B","PLXDC1","IKZF3","GFOD1","CRTAM","DUSP6","RGS1","TP53I11","GFI1","IFNG","SLC7A5","GCNT4")
human_CD4_CD69_down<-c("FAM65B","STK38","GRASP","KLF3","SAMD3","GABBR1","FRY","ARHGEF11","VIPR1","BAIAP3","MFGE8","SBK1","HAPLN3","TTC16","CX3CR1","USP46","PLXNA4","D4S234E","DSEL","CNTNAP1","VSIG1","RGMB","TTYH2","EPHA4","TNFRSF11A","MUC1","CR1","E2F2","KLF2","EDA","KRT73","ZNF462","RAP1GAP2","S1PR1","NPDC1","FLJ13197","ISM1","TSPAN18","KCTD15","KRT72","SEMA5A","WNT7A","SOX13","FUT7","PTGDS","PI16","SEMA3G","SYT4")
human_CD4_CD69_up_expressed <- intersect(human_CD4_CD69_up, rownames(seu))
human_CD4_CD69_down_expressed <- intersect(human_CD4_CD69_down, rownames(seu))


human_CD8_CD69_up<-c("CD69","CA10","IL17A","CXCL13","SCUBE1","GSG2","ITGA1","CXCR6","ATP8B4","CSF1","ITGAE","CPNE7","IL10","SPRY1","MCAM","RGS1","KCNQ3","DAB2IP","TRPM2","KCNK5","IL23R","PELO","COL5A1","IRF4","FSD1","IL17RE","ADAM12","CRTAM","ARHGAP18","CCR1","AMICA1","ICOS","TMIGD2","TP53INP1","BMF","CD9","RIMS3","DUSP6","CCR6","GZMB","ZNF683")
human_CD8_CD69_down<-c("PXN","FLNA","CYB561","CD300A","TSPAN32","RASA3","GPR114","TGFBR3","SAMD3","PELI2","C11orf21","RASGRP2","SYNE1","GK5","SSX2IP","STK38","FGR","SSBP3","CFH","ADAMTS10","MTSS1","KLF3","KLF2","SVIL","CACNA2D2","FAM65B","SBK1","PATL2","TMCC3","KIR2DS4","HPCAL4","VCL","TTC16","PDZD4","DCHS1","EBF4","OSBPL5","FZD4","GNLY","NHSL2","TSPAN18","ME3","MSX2P1","ZNF711","D4S234E","FCGR3A","GPA33","COL6A2","CXCR2","TTYH2","AGPAT4","TKTL1","SELP","LILRB1","ITGAM","LOXL4","FLJ13197","TFCP2L1","C1orf21","SLCO4C1","NUAK1","PALLD","DNAI2","SOX13","S1PR1","SELL","PLEKHG3","GPR56","SPTB","ZNF365","PCDH1","NPDC1","KRT73","KRT72","ASCL2","FAM19A1","SGCD","LAIR2","EFHC2","RAP1GAP2","TXNDC3","PODN","SH3RF2","KIF19","PTGDS","EPHX4","PRSS23","KIR3DX1","CX3CR1","SLC1A7","FGFBP2","LRFN2","DGKK")
human_CD8_CD69_up_expressed <- intersect(human_CD8_CD69_up, rownames(seu))
human_CD8_CD69_down_expressed <- intersect(human_CD8_CD69_down, rownames(seu))

Proinflam_cytokines_human<-c(
    #'IL1',
    'IL2',
    #'IL6',
    #'IL1B',
    'TNF',
    'IFNG',
    'IL17A',
    'IL17F',
    'IL22')
Proinflam_cytokines_human_expressed <- intersect(Proinflam_cytokines_human, rownames(seu))

In [ ]:
Proinflam_cytokines_human_expressed

In [ ]:
for (n in Proinflam_cytokines_human_expressed){
    print(FeaturePlot(object = seu,features = n,
                  cols = c("lightgrey", "blue"),
                      order = T,
                      slot = "data",
                      #min.cutoff = "q05", max.cutoff = "q95",
                      reduction = "umap",
                pt.size=0.1
                       ))   
      print(VlnPlot(object = seu, features = n,
                 #group.by = "sample",
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0.01
                      )+NoLegend()
   )
          print(VlnPlot(object = seu, features = n,
                 #group.by = "sample",
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )+NoLegend()
   )
   #print(RidgePlot(seu, 
   #          features = n))

    }

In [ ]:
Proinflam_cytokines_human_expressed

In [ ]:
Trm_genes=list(human_Trm_gene_up_expressed,
               human_Trm_gene_down_expressed,
               human_CD4_CD69_up_expressed,
               human_CD4_CD69_down_expressed,
               human_CD8_CD69_up_expressed,
               human_CD8_CD69_down_expressed,
               Proinflam_cytokines_human_expressed
              )

#CD4Trm_genes=list(human_CD4_CD69_up_expressed,human_CD4_CD69_down_expressed)

#CD4Trm_genes=list(human_Trm_gene_up_expressed,human_Trm_gene_down_expressed)

seu<-AddModuleScore(seu, features = Trm_genes, pool = NULL, nbin = 24,
  ctrl = 100, k = FALSE, assay = NULL, name = c("Trm_score",
                                                "non_Trm_score",
                                               "CD4_Trm_score",
                                                "CD4_non_Trm_score",
                                                "CD8_Trm_score",
                                                "CD8_non_Trm_score",
                                                "Proinflam_cytokines_score"
                                               ), seed = 1)
head(seu@meta.data)


In [ ]:
TRM_scores<-c("Trm_score1",
 "non_Trm_score2",
#"CD4_Trm_score3",
# "CD4_non_Trm_score4",
# "CD8_Trm_score5",
 #"CD8_non_Trm_score6",
 "Proinflam_cytokines_score7")

In [ ]:
for (n in TRM_scores){
    print(FeaturePlot(object = seu,features = n,
                  cols = c("lightgrey", "blue"),
                      order = T,
                      slot = "data",
                      min.cutoff = 0, 
                      max.cutoff = 0.8, 
                      #max.cutoff = "q95",
                      reduction = "umap",
                pt.size=0.1
                       ))   
      print(VlnPlot(object = seu, features = n,
                 #group.by = "sample",
                    #cols = cols
                    split.by = "tissue",
                    split.plot = TRUE,
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )
   )
    print(VlnPlot(object = seu, features = n,
                  cols = cols,
                 #group.by = "sample",
                    #split.by = "tissue",
                    #split.plot = TRUE,
                    y.max = 0.8,
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )
    )
        print(VlnPlot(object = seu, features = n,
                 group.by = "tissue",
                 #cols = cols,
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )+NoLegend()
   )
    }

In [ ]:
#seu_1<-seu

In [ ]:
Th1_genes_human<-c("CD4","CXCR3","TBX21","IFNG","TNF")
Th17_genes_human<-c("CD4","CCR6","IL23R","RORC","RORA","IL17A","IL17F","IL22")
Th2_genes_human<-c("CD4",
                   "GATA3","IL4","IL5","IL13")
Th1_genes_human_expressed <- intersect(Th1_genes_human, rownames(seu))
Th17_genes_human_expressed <- intersect(Th17_genes_human, rownames(seu))
Th2_genes_human_expressed <- intersect(Th2_genes_human, rownames(seu))

In [ ]:
Th_genes=list(Th1_genes_human_expressed,
               Th17_genes_human_expressed,
               Th2_genes_human_expressed
              )

In [ ]:
seu_1<-AddModuleScore(seu_1, features = Th_genes, pool = NULL, nbin = 24,
  ctrl = 100, k = FALSE, assay = NULL, name = c("Th1_score",
                                                "Th17_score",
                                               "Th2_score"
                                               ), seed = 1)
head(seu_1@meta.data)

In [ ]:
Th_scores<-c("Th1_score1",
 "Th17_score2",
"Th2_score3")
for (n in Th_scores){
    print(FeaturePlot(object = seu_1,features = n,
                  cols = c("lightgrey", "blue"),
                      order = T,
                      slot = "data",
                      min.cutoff = 0,
                      max.cutoff = 0.8, 
                      #max.cutoff = "q95",
                      #max.cutoff = 1,
                      reduction = "umap",
                pt.size=0.1
                       ))   
      print(VlnPlot(object = seu_1, features = n,
                 #group.by = "sample",
                    split.by = "tissue",
                    split.plot = TRUE,
                    y.max = 0.8,
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )
   )
    print(VlnPlot(object = seu_1, features = n,
                  cols = cols,
                 #group.by = "sample",
                    #split.by = "tissue",
                    #split.plot = TRUE,
                    y.max = 0.8,
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )
   )
        print(VlnPlot(object = seu_1, features = n,
                 group.by = "tissue",
                      y.max = 0.8,
                 #x.lab.rot=T, 
                 #size.x.use = 5,
                 pt.size = 0
                      )+NoLegend()
   )
    }

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive ones
seu.markers <- FindAllMarkers(object = seu, only.pos = TRUE, 
                              min.pct = 0.25, logfc.threshold = 0.25)
#logfc.threshold=0.25 (default) instead of old version thresh.use=0.25
head(seu.markers)
dim(seu.markers)

In [ ]:
top3 <- seu.markers %>% group_by(cluster) %>% top_n(3, avg_log2FC)
top5 <- seu.markers %>% group_by(cluster) %>% top_n(5, avg_log2FC)
top10 <- seu.markers %>% group_by(cluster) %>% top_n(10, avg_log2FC)
top10

In [ ]:
plt1<-DotPlot(seu, features = rev(unique(top10$gene)),
              dot.scale = 5
              #scale.by = "size"
             ) + coord_flip()+
theme(#strip.background = element_blank(),
    axis.title = element_blank(),
    axis.text.x = element_text(size = 10,angle = 45,hjust = 1),
    axis.text.y = element_text(size = 10),
     legend.position = "right",
     #legend.spacing = unit(0, "mm"),
     legend.direction = "vertical",
        legend.text = element_text(size=5),
        legend.key.width = unit(2, "mm"),
        legend.key.height = unit(2, "mm"),
        legend.box.spacing = unit(1, "mm"),
        legend.margin = margin(2),
        legend.title = element_text(size = 7,angle = 90)
)
plt1

In [ ]:
levels(seu)

In [ ]:
df<-seu@meta.data[,c("sample","case","tissue","cell_type")]
dim(df)
head(df)

In [ ]:
df$cell_type <- factor(df$cell_type,levels = rev(my_levels))

In [ ]:
#levels(df$cell_type)<-my_levels

In [ ]:
cols

In [ ]:
plt<-ggplot(df, aes(x=cell_type, fill=tissue)) +
geom_bar(position = "fill",linetype = "solid", colour = "black")+
geom_errorbar(data=df, aes(x=cell_type,
                           ymax = table(df$tissue)[length(table(df$tissue))]/nrow(df),
                           ymin = table(df$tissue)[length(table(df$tissue))]/nrow(df)),
               size=0.5, linetype = "longdash", 
              inherit.aes = F, width = 1) +
geom_errorbar(data=df, aes(x=cell_type,
                           ymax = (1-as.numeric(table(df$tissue)[1]/nrow(df))),
                           ymin = (1-as.numeric(table(df$tissue)[1]/nrow(df)))),
               size=0.5, linetype = "longdash", 
              inherit.aes = F, width = 1) +
#scale_fill_manual(values=c("green","red"),labels=c("K","B"))+
coord_flip()
plt+theme(axis.text.x = element_text(angle = 0, hjust = 0.5,colour="black", size = 12),
     axis.text.y=element_text(colour="black", size = 12),
     legend.position="bottom", legend.direction="horizontal", legend.title = element_blank()
    )+ labs(x="",y="Proportion")+guides(fill = guide_legend(reverse = TRUE))

In [ ]:
plt<-ggplot(df, aes(x=cell_type, fill=tissue)) +
geom_bar(position = "fill",linetype = "solid", colour = "black")+
geom_errorbar(data=df, aes(x=cell_type,
                           ymax = table(df$tissue)[length(table(df$tissue))]/nrow(df),
                           ymin = table(df$tissue)[length(table(df$tissue))]/nrow(df)),
               size=0.5, linetype = "longdash", 
              inherit.aes = F, width = 1) +
geom_errorbar(data=df, aes(x=cell_type,
                           ymax = (1-as.numeric(table(df$tissue)[1]/nrow(df))),
                           ymin = (1-as.numeric(table(df$tissue)[1]/nrow(df)))),
               size=0.5, linetype = "longdash", 
              inherit.aes = F, width = 1) #+
#scale_fill_manual(values=c("green","red"),labels=c("DN","Ctrl"))+
#coord_flip()
plt+theme(axis.text.x = element_text(angle = 45, hjust = 1,colour="black", size = 12),
     axis.text.y=element_text(colour="black", size = 12),
     legend.position="bottom", legend.direction="horizontal", legend.title = element_blank()
    )+ labs(x="",y="Proportion")+guides(fill = guide_legend(reverse = TRUE))

In [ ]:
#ggplot(df, aes(x=seurat_clusters, fill=tissue))

myplot <- ggplot(df, aes(cell_type, 
                         group = tissue#,
                         #fill=cols
                        )) + 
          geom_bar(aes(y = ..prop.., fill = factor(..x..)
                      ), stat="count") +
coord_flip()+ 
          scale_y_continuous(labels=scales::percent) +
          ylab("relative frequencies") +
          scale_fill_manual(values=rev(cols),labels=rev(my_levels))+
          theme(axis.text.x = element_text(angle = 0, hjust = 1,colour="black", size = 10),
               axis.text.y = element_text(size = 15))+
          facet_grid(~tissue)+NoLegend()
myplot

In [ ]:
#saveRDS(seu_sub, file = paste0("Integrated_objects/",samples_ID,"_CD4EMRM.rds"))

In [ ]:
DimPlot(object = seu, reduction = 'umap',label = TRUE,
        cols = cols,
        pt.size = 0.1,label.size = 4, repel = T
       ) + theme(aspect.ratio=1,legend.position = "right")

In [ ]:
samples_ID

In [ ]:
saveRDS(seu, file = paste0("Integrated_objects/",samples_ID,"_celltype.rds"))